In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU
from sklearn.model_selection import train_test_split

2024-06-10 11:20:39.395373: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 11:20:40.583218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [28]:
data = pd.read_csv('../data/jena_climate_2009_2016.csv', index_col="Date Time")
data.index = pd.to_datetime(data.index)

In [29]:
data.head(5)

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [52]:
train, _temp = train_test_split(data, test_size=0.2, shuffle=False)
val, test = train_test_split(_temp, test_size=0.25, shuffle=False)

In [53]:
#scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()

scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_val = scaler.transform(val)
scaled_test = scaler.transform(test)

In [57]:
X_train, y_train = [], []
k = 120

for i in range(k,len(scaled_train)):
    X_train.append(scaled_train[i-k:i, :])
    y_train.append(scaled_train[i, 2])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [59]:
X_val, y_val = [], []
k = 120

for i in range(k,len(scaled_val)):
    X_val.append(scaled_val[i-k:i, :])
    y_val.append(scaled_val[i, 2])

X_val = np.array(X_val)
y_val = np.array(y_val)

In [60]:
X_test, y_test = [], []
k = 120

for i in range(k,len(scaled_test)):
    X_test.append(scaled_test[i-k:i, :])
    y_test.append(scaled_test[i, 2])

X_test = np.array(X_test)
y_test = np.array(y_test)

In [65]:
X_train.shape[1]

120

In [66]:
model = Sequential()
model.add(LSTM(units=60, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=60))
model.add(Dense(1,activation = 'sigmoid'))
model.add(Dense(1,activation = 'linear'))
model.summary()

2024-06-10 13:04:14.717291: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 13:04:14.984505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 13:04:14.984686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 13:04:14.992448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:65:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 13:04:14.992553: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 120, 60)           18000     
                                                                 
 lstm_1 (LSTM)               (None, 60)                29040     
                                                                 
 dense (Dense)               (None, 1)                 61        
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 47103 (184.00 KB)
Trainable params: 47103 (184.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_absolute_error')

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    validation_data=(X_val, y_val),
    validation_batch_size=32,
    epochs=4,
    callbacks=[],
    verbose=1)